In [ ]:
#planner
from pydantic import BaseModel, Field
from agents import Agent
from datetime import datetime

HOW_MANY_SEARCHES = 5

INSTRUCTIONS = f"You are a research assistant for a company. \
Given the company name, industry and a research query, come up with a set of web searches to perform to best answer the query considering areas such as \
user or customer feedback, competitor analytics, market trends, etc. The current date is {datetime.now().strftime('%Y-%m-%d')}. \
Output {HOW_MANY_SEARCHES} terms to query for.  Ensure the search queries are specific to the company."

class WebSearchItem(BaseModel):
    reason: str = Field(description="Your reasoning for why this search is important to the query.")
    query: str = Field(description="The search term to use for the web search.")

class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem] = Field(description="A list of web searches to perform to best answer the query.")

planner_agent = Agent(
    name="PlannerAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=WebSearchPlan
)

In [ ]:
#searcher
from agents import Agent, WebSearchTool, ModelSettings

INSTRUCTIONS = (
    f"You are a research assistant for a company. \
    Given a search term, you search the web for that term and produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 "
    "words. Capture the main points. Write succintly, no need to have complete sentences or good "
    "grammar. This will be consumed by someone synthesizing a report, so its vital you capture the "
    "essence and ignore any fluff. Do not include any additional commentary other than the summary itself."
)

search_agent = Agent(
    name="Search agent",
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required")
)

In [ ]:
#writer
from pydantic import BaseModel, Field
from agents import Agent

INSTRUCTIONS = (
    f"You are a senior researcher for a company.  You are tasked with writing a cohesive report for a research query. \
    You will be provided with the company name, industry, original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 5-10 pages of content, at least 1000 words."
)

class ReportData(BaseModel):
    short_summary: str = Field(description="A short 2-3 sentence summary of the findings.")

    markdown_report: str = Field(description="The final report")

    follow_up_questions: list[str] = Field(description="Suggested topics to research further")


writer_agent = Agent(
    name="WriterAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=ReportData
)

In [ ]:
import os
from typing import Dict
import smtplib
from email.mime.text import MIMEText
from agents import Agent, function_tool

@function_tool
def send_email(subject: str, html_body: str, email_address: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to the user """

    # Your Gmail credentials
    gmail_user = os.getenv("FROM_EMAIL") 
    gmail_app_password = os.getenv("GOOGLE_APP_PW")
    to = email_address #os.getenv("TO_EMAIL") 

    if not all([gmail_user, gmail_app_password, to]):
        raise ValueError("Missing one or more required environment variables: FROM_EMAIL, GOOGLE_APP_PW, TO_EMAIL")

    # Create the email
    msg = MIMEText(html_body, "html")
    msg['Subject'] = subject or ""
    msg['From'] = gmail_user or ""
    msg['To'] = to or ""

    # Send the email
    try:
        with smtplib.SMTP('smtp.gmail.com', 587) as server:
            server.starttls()  # Secure the connection
            server.login(gmail_user, gmail_app_password)
            server.send_message(msg)
        print('Email sent successfully!')
        return {"status": "success"}
    except Exception as e:
        print(f'Failed to send email: {e}')


#handoff prompt recommended by openai
RECOMMENDED_PROMPT_PREFIX = "# System context\nYou are part of a multi-agent system called the Agents SDK, designed to make agent coordination and execution easy. \
Agents uses two primary abstraction: **Agents** and **Handoffs**. An agent encompasses instructions and tools and can hand off a conversation to another agent when \
appropriate. Handoffs are achieved by calling a handoff function, generally named `transfer_to_<agent_name>`. Transfers between agents are handled seamlessly in the background;\
do not mention or draw attention to these transfers in your conversation with the user.\n"

INSTRUCTIONS = f"""{RECOMMENDED_PROMPT_PREFIX} You are able to send a nicely formatted HTML email based on a detailed report.
You will be provided with a detailed report and the email address to which it should be sent. You should use your tool to send one email, providing the 
report converted into clean, well presented HTML with an appropriate subject line."""

email_agent = Agent(
    name="Email agent",
    instructions=INSTRUCTIONS,
    tools=[send_email],
    model="gpt-4o-mini",
    handoff_description="Convert a report to HTML and send it"
)

In [ ]:
#research manager
from agents import Agent, Runner, trace, gen_trace_id, SQLiteSession, handoff, trace
from agents.extensions import handoff_filters


# Convert agents to tools
planner_tool = planner_agent.as_tool(tool_name="planner_agent", tool_description="Create search strategy")
search_tool = search_agent.as_tool(tool_name="search_agent", tool_description="Execute web searches and summarises results")
writer_tool = writer_agent.as_tool(tool_name="writer_agent", tool_description="Generate research report")

# Research Manager Agent

#handoff prompt recommended by openai
RECOMMENDED_PROMPT_PREFIX = "# System context\nYou are part of a multi-agent system called the Agents SDK, designed to make agent coordination and execution easy. \
Agents uses two primary abstraction: **Agents** and **Handoffs**. An agent encompasses instructions and tools and can hand off a conversation to another agent when \
appropriate. Handoffs are achieved by calling a handoff function, generally named `transfer_to_<agent_name>`. Transfers between agents are handled seamlessly in the background;\
do not mention or draw attention to these transfers in your conversation with the user.\n"

INSTRUCTIONS = (
    f"""{RECOMMENDED_PROMPT_PREFIX} You research companies and create reports. You have these tools:
- planner_tool: Create search strategies
- search_tool: Find information  
- writer_tool: Create reports

Look at the conversation history to understand what's needed:
- New research request? Plan, search, write.
- Feedback on existing research? Update appropriately.
- Email request? Transfer to email agent.

Use your intelligence to decide which tools to use and when.

Always output only the final markdown report - no commentary or explanations.
    """
)

#simplify handoff
email_handoff = handoff(
    agent=email_agent,
    input_filter=handoff_filters.remove_all_tools,  # This removes all tool calls from history
    tool_description_override="Send the research report via email to the specified email address"
)

research_manager_agent = Agent(
    name="Research Manager",
    instructions=INSTRUCTIONS,
    model="gpt-4o",
    handoffs=[email_handoff],
    tools=[planner_tool, search_tool, writer_tool]
)

In [ ]:
#research function

session_store = {}

async def run_research(company: str, industry: str, query: str, feedback: str, email_trigger: str):
    
    #session handling - to ensure session persists    
    session_id = f"research_{company}_{query}"
    if session_id not in session_store:
        session_store[session_id] = SQLiteSession(session_id)
    session = session_store[session_id] 
  
    if hasattr(session, 'trace_id') and session.trace_id:
        trace_id = session.trace_id
    else:
        trace_id = gen_trace_id()
        session.trace_id = trace_id
    
    with trace("Research trace", trace_id=trace_id):
        print(f"View trace: https://platform.openai.com/traces/trace?trace_id={trace_id}")

        if not feedback and not email_trigger:
            # Initial research
            yield "Researching..."
            
            result = await Runner.run(
                research_manager_agent, 
                f"Research: Company: {company} | Industry: {industry} | Query: {query}",
                session=session
            )
            yield result.final_output
        
        elif feedback and not email_trigger:
            # Feedback processing
            yield "Processing feedback..."
            
            result = await Runner.run(
                research_manager_agent, 
                f"Based on your previous research, here is user feedback: {feedback}\n\n\
                Please update and improve the existing research report based on this feedback. Do not start over - build upon what you already provided.",
                session=session
            )
            yield result.final_output

        elif email_trigger.startswith("EMAIL_REPORT"):
            
            yield "Preparing to email report..."
            
            email_address = email_trigger.split("Email address: ")[1]
            result = await Runner.run(
                research_manager_agent,
                f"The user wants to email the report. Please hand off to the emailer agent to send the final research report and share the email: {email_address}",
                session=session
            )
            yield result.final_output

        else:
            yield "The research process is complete.  Please clear and start again."

In [ ]:
# functions for gradio
import gradio as gr
import re

#email functions
def trigger_email(email):
    return f"EMAIL_REPORT, Email address: {email}"

def show_email_fields():
    return gr.update(visible=True,interactive=True), gr.update(visible=True), gr.update(visible=True,interactive=True)

def validate_email(email):
    "Validates an email address using a regular expression."
    
    if not email:
        return "Email cannot be empty."
    pattern = r"^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$"
    if re.match(pattern, email):
        return "Valid email address."
    else:
        return "Invalid email address format."

#main function
#first run
async def run_initial(company: str, industry: str, query: str):

    # Validation
    if not company or not company.strip():
        raise gr.Error("Please enter a company name")
    if not industry or not industry.strip():
        raise gr.Error("Please enter an industry")
    if not query or not query.strip():
        raise gr.Error("Please enter a research query")

    async for chunk in run_research(company, industry, query, None, None):
        yield chunk

#feedback run
async def run_feedback(company: str, industry: str, query: str, feedback: str):
    # Validation
    if not feedback or not feedback.strip():
        raise gr.Error("Please enter your feedback")

    async for chunk in run_research(company, industry, query, feedback, None):
        yield chunk

#email handoff run
async def run_email(company: str, industry: str, query: str, feedback: str, email_trigger: str):
    async for chunk in run_research(company, industry, query, feedback, email_trigger):
        yield chunk


In [ ]:
# final run and gradio
import gradio as gr

with gr.Blocks(theme=gr.themes.Ocean()) as ui:
    gr.Markdown("Deep Research for Companies")
    gr.Markdown("Generate comprehensive company research reports with AI-powered analysis")
    
    with gr.Row():
        with gr.Column(scale=1):
            org_textbox = gr.Textbox(label="Organisation Name",placeholder="e.g. ComplAI",info="The company you want to research")
            industry_textbox = gr.Textbox(label="Industry", placeholder="e.g. Finance, Healthcare, SaaS",info="Primary industry or sector")
            query_textbox = gr.Textbox(label="Research Topic",placeholder="e.g. Analyze competitor pricing strategies",lines=3,info="What specific aspect would you like to research?")
            
            with gr.Row():
                run_button = gr.Button("Start Research", variant="primary", size="lg")

        with gr.Column(scale=1):
            feedback_textbox = gr.Textbox(label="Feedback", placeholder="e.g. Can you provide more information on...?",lines=3,info="Provide feedback to improve the research")
            email_trigger_textbox = gr.Textbox(label="Email trigger", visible=False)
            feedback_button = gr.Button("Provide Feedback", variant="primary",scale=1)
            send_button = gr.Button("Email me the report", variant ="secondary",scale=1)
            with gr.Column(scale=1):
                email_address = gr.Textbox(label="Email address:",type="email",visible=False,interactive=True)
                email_validation = gr.Markdown(visible=False)
                confirm_send_button = gr.Button("Go, please send!", variant ="primary",visible=False)
        
        with gr.Column(scale=2):
            report = gr.Markdown(label="Research Report",value="Your research report will appear here...",height=600)
    
    # Event handlers
    run_button.click(fn=run_initial, inputs=[org_textbox, industry_textbox, query_textbox], outputs=report, show_progress=True)
    feedback_button.click(fn=run_feedback, inputs=[org_textbox, industry_textbox, query_textbox, feedback_textbox], outputs=report, show_progress=True)
    send_button.click(fn=show_email_fields,inputs=[],outputs=[email_address, email_validation, confirm_send_button])
    confirm_send_button.click(fn=validate_email, inputs=email_address, outputs=email_validation).then(fn=trigger_email, inputs=email_address, outputs=email_trigger_textbox).then(
    fn=run_email, inputs=[org_textbox, industry_textbox, query_textbox, feedback_textbox, email_trigger_textbox], outputs=report)
        
    gr.ClearButton(components=[org_textbox,industry_textbox,query_textbox,feedback_textbox,report,email_address],value="Clear all fields")

ui.launch(inbrowser=True, share=False)

